In [49]:
import sys
sys.path.append('../')
import string
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from pprint import pprint
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.utils import to_categorical
import keras.backend as K
from gensim.models import KeyedVectors
import word2vecReader as godin_embedding
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [12,10]
from mlens.visualization import corrmat

In [30]:
def load_data_from_file(filename,test_flag = False):
    data = pd.read_csv(filename, sep="\t", header=None)
    if not test_flag:
        data.columns = ["tweet_id", "username", "database_id", "class","tweet"]
    else:
        data.columns = ["a", "b", "med","med", "tweet","class",]
    return data

In [31]:
train_data = load_data_from_file('dataset/personal_intake_tweets.txt')
dev_data = load_data_from_file('dataset/personal_intake_tweets_dev.txt')

In [32]:
train_sentences = train_data['tweet'].tolist()+dev_data['tweet'].tolist()
train_labels = train_data['class'].tolist()+dev_data['class'].tolist()

In [33]:
test_data = load_data_from_file('dataset/task_2_test_full_form.txt',test_flag=True)

In [34]:
test_labels = test_data['class'].tolist()
test_sentences = test_data['tweet'].tolist()

In [35]:
len(train_labels),len(train_sentences),len(test_labels),len(test_sentences)

(9107, 9107, 7419, 7419)

In [36]:
test_labels = [x-1 for x in test_labels]
train_labels = [x-1 for x in train_labels]

In [37]:
number_of_classes = len(set(train_labels))
number_of_classes

3

In [38]:
def remove_punctuation(s):
    list_punctuation = list(string.punctuation)
    for i in list_punctuation:
        s = s.replace(i,'')
    return s

In [39]:
def clean_sentence(sentence):
    #removes links
    sentence = re.sub(r'(?P<url>https?://[^\s]+)', r'', sentence)
    # remove @usernames
    sentence = re.sub(r"\@(\w+)", "", sentence)
    #remove # from #tags
    sentence = sentence.replace('#','')
    # split into tokens by white space
    tokens = sentence.split()
    # remove punctuation from each token
    # should have used translate but for some reason it breaks on my server
    tokens = [remove_punctuation(w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    tokens = ' '.join(tokens)
    return tokens

In [40]:
print("cleaning data")
trainX = [clean_sentence(s) for s in train_sentences]
testX = [clean_sentence(s) for s in test_sentences]
trainY = np.array(train_labels)

cleaning data


In [41]:
max_len = 20

In [42]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [43]:
def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [44]:
def load_godin_word_embedding(path):
    print("Loading Goding model.")
    return godin_embedding.Word2Vec.load_word2vec_format(path, binary=True)

In [45]:
def get_word_embedding_matrix(model,dim):
    #dim = 300 for google word2vec
    #dim = 400 for godin
    #dim = 100 for fast text
    embedding_matrix = np.zeros((vocab_size,dim))
    for word, i in tokenizer.word_index.items():
        try:
            embedding_vector = model[word]
        except KeyError:
            embedding_vector = None
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector
    return embedding_matrix

In [46]:
def get_results(test_labels,pred_class):
    f1 = f1_score(test_labels,pred_class,labels=[0,1],average='micro')
    p = precision_score(test_labels,pred_class,labels=[0,1],average='micro')
    r = recall_score(test_labels,pred_class,labels=[0,1],average='micro')
    acc = accuracy_score(test_labels,pred_class)
    return [f1,p,r,acc]

In [47]:
def get_pred_class(model):
    pred = model.predict(testX)
    return [np.argmax(x) for x in pred]

In [48]:
tokenizer = create_tokenizer(trainX)
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % max_len)
print('Vocabulary size: %d' % vocab_size)
trainX = encode_text(tokenizer, trainX, max_len)
testX = encode_text(tokenizer, testX, max_len)
trainY = to_categorical(trainY,num_classes=number_of_classes)

Max document length: 20
Vocabulary size: 10940


In [21]:
# godin_model = load_godin_word_embedding("../word_embeddings/word2vec_twitter_model.bin")

In [22]:
# embedding_matrix_godin = get_word_embedding_matrix(godin_model,400)

## Base Learners

In [23]:
model_cnn = load_model('models/cnn.h5')
model_bi_lstm = load_model('models/bi_lstm.h5')
model_cnn_bi_lstm = load_model('models/cnn_bi_lstm.h5')
model_cnn_lstm = load_model('models/cnn_lstm.h5')

In [26]:
base_models = [('cnn',model_cnn),('bi_lstm',model_bi_lstm),('cnn_bi_lstm',model_cnn_bi_lstm),('cnn_lstm',model_cnn_lstm)]

In [28]:
get_pred_class(model_cnn)

InvalidArgumentError: indices[6,0] = 10914 is not in [0, 10906)
	 [[Node: embedding_1/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_1/embeddings/read, embedding_1/Cast)]]

Caused by op 'embedding_1/Gather', defined at:
  File "/home/hitkul/anaconda3/envs/ps3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hitkul/anaconda3/envs/ps3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File 

In [25]:
pred_class_base = [(name,get_pred_class(m)) for name,m in base_models]

InvalidArgumentError: indices[6,0] = 10914 is not in [0, 10906)
	 [[Node: embedding_1/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_1/embeddings/read, embedding_1/Cast)]]

Caused by op 'embedding_1/Gather', defined at:
  File "/home/hitkul/anaconda3/envs/ps3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hitkul/anaconda3/envs/ps3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File 

In [ ]:
pred_mat = np.zeros((len(pred_class_base[0][1]),len(pred_class_base)),dtype = np.int64)
for i,p in enumerate(pred_class_base):
    pred_mat[:,i] = p[1]

pred_df = pd.DataFrame(pred_mat)
pred_df.columns = ["cnn", "bi_lstm","cnn_bi_lstm",,"cnn_lstm"]

In [ ]:
result_base = [(name,get_results(test_labels,pred_class)) for name,pred_class in pred_class_base]
for i,t in enumerate(result_base):
    result_base[i] = t[1].insert(0,t[0])

In [ ]:
model_performance = pd.DataFrame(columns=['model','f1', 'precision', 'recall','accuracy'])

In [ ]:
for i,result in enumerate(result_base):
    model_performance.loc[i] = result